In [8]:
import time
from collections import deque
from random import choice
IP_POOL = ['192.168.1.1', '192.168.1.2', '192.168.1.3', '192.168.1.4', '192.168.1.5'] ##ip's for rotation

#helper function to simulate making an HTTP request 
def simulate_http_request(ip):
    print(f"Request made from IP: {ip}")

#class for throttling mechanism (to be implemented) 
class Throttler:
    def __init__(self, rate_limit):
        self.requests = deque()  
        self.rate_limit = rate_limit 

    def allow_request(self):
        current_time = time.time()
        # if more than 1 min old
        while self.requests and current_time - self.requests[0] > 60:
            self.requests.popleft()
        
        # if new request more than rate limit
        if len(self.requests) < self.rate_limit:
            self.requests.append(current_time)
            return True
        else:
            return False

##function to be implemented
def make_requests(n):
    """ 
    Implement this function to make `n` requests to the server using IP rotation and a custom throttling 
    mechanism. 

    Args: - n (int): Number of requests to make. 
    Rotate through the IP_POOL list for each request and ensure requests are throttled to no more than 5 
    requests per minute. 
    Use the `simulate_http_request` function to make a request, selecting the next available IP from 
    `IP_POOL` in a round-robin manner. 
    """ 
    throttler = Throttler(5) 
    ip_index = 0
    
    for _ in range(n):
        if ip_index >= len(IP_POOL):
            ip_index = 0
        ip = IP_POOL[ip_index]
        ip_index += 1

        while not throttler.allow_request():
            time.sleep(1)  # Wait a bit if the rate limit is hit
            
        # Make request
        simulate_http_request(ip)

make_requests(10)

Request made from IP: 192.168.1.1
Request made from IP: 192.168.1.2
Request made from IP: 192.168.1.3
Request made from IP: 192.168.1.4
Request made from IP: 192.168.1.5
Request made from IP: 192.168.1.1
Request made from IP: 192.168.1.2
Request made from IP: 192.168.1.3
Request made from IP: 192.168.1.4
Request made from IP: 192.168.1.5


Question 2

In [3]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [6]:
def load_images_from_folder(folder, max_images_per_class=1000):
    # changed the code a bit to have a limit on 
    # max images we are reading per class as
    # currently its very computationally expensive
    # to train SVM on full 25000 images
    images = []
    labels = []
    class_count = {'cat': 0, 'dog': 0}  # Dictionary to track the count of each class

    for filename in os.listdir(folder):
        current_class = 'cat' if 'cat' in filename else 'dog'
        if class_count[current_class] >= max_images_per_class:
            continue
        img = imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            # Consider handling label transformation within the training
            # and evaluation functions if necessary (typically, SVM labels are -1 and 1)
            # so we are transforming labels to -1 for cat and 1 for dog
            labels.append(-1 if 'cat' in filename else 1)
            class_count[current_class] += 1
    return images, labels

def preprocess_images(images, target_size=(64, 64)): 
    """ 
    Resize images and flatten them for SVM processing. 
     
    Args: 
    - images (list of ndarray): List of images. 
    - target_size (tuple of int): Size to which images will be resized. 
 
    Returns: 
    - np.array: Array of reshaped images. 
    """ 
    processed_images = [] 
    for img in images: 
        img_resized = resize(img, target_size, anti_aliasing=True, mode='reflect') 
        processed_images.append(img_resized.flatten()) 
    return np.array(processed_images)

# training on 2 epochs only in interest of time
def train_svm(features, labels, epochs=10, learning_rate=0.01, lambda_param=0.01): 
    """ 
    Train a linear SVM classifier from scratch. 
     
    Args: 
    - features (np.array): Feature array of image data. 
    - labels (list of int): List of labels corresponding to the images. 
    - epochs (int): Number of epochs to train. 
    - learning_rate (float): Learning rate for the gradient descent. 
    - lambda_param (float): Regularization parameter. 
 
    Returns: 
    - np.array: Weight vector after training. 
    """ 
    weights = np.zeros(features.shape[1])
    for epoch in range(epochs):
        for i, feature in enumerate(features):
            condition = labels[i] * np.dot(weights, feature)
            if condition < 1:
                # if incorrect prediction update weights
                weights -= learning_rate * ((2 * lambda_param * weights) - labels[i] * feature)
            else:
                # or if Correct prediction then just apply regularization
                weights -= learning_rate * (2 * lambda_param * weights)
    return weights

def evaluate_svm(weights, features, labels): 
    """ 
    Evaluate the trained SVM on a test set. 
     
    Args: 
    - weights (np.array): The weights of the trained SVM. 
    - features (np.array): Feature array of image data for testing. 
    - labels (list of int): List of labels corresponding to the test images. 
 
    Returns: 
    - float: Accuracy of the classifier on the test set. 
    """ 
    predictions = np.sign(np.dot(features, weights))
    accuracy = np.mean(predictions == labels)
    return accuracy

##load and preprocess images 
folder = 'tempimage' # i have pasted 1127 images of cat and 1103 images of dogs in tempimages folder and using that folder im working 
images, labels = load_images_from_folder(folder) 
processed_images = preprocess_images(images) 

##split data into training and testing sets 
features_train, features_test, labels_train, labels_test = train_test_split( 
    processed_images, labels, test_size=0.2, random_state=42)

weights = train_svm(features_train, labels_train) 
accuracy = evaluate_svm(weights, features_test, labels_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4925


In this we can say that instead of just taking pixel values of image you can use image features such as SIFT, SURF, edge and corner detectors or some modern img feature techniques from CNNs etc. Along witht this we can change the SVM kernel to a higher dimensional space if data isn't linearly separable in low dimension